In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

### 검색기능 SERPAPI 활용

- 강사님께 도움을 받았습니다.

In [ ]:
from langchain_community.utilities import SerpAPIWrapper

# 구글 검색엔진, 검색 결과의 지역을 한국,언어를 한국어, 검색 결과를 3개만 반환
params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

# 1. SerpAPIWrapper 인스턴스 생성
search = SerpAPIWrapper(params=params)

In [34]:
# 2. run 메서드를 사용하여 검색 실행
# 검색하고 싶은 키워드를 전달합니다.
query = "5호선 상일동역 주변에 있는 맛집"
search_result = search.run(query=query)

In [35]:
search_results = eval(search_result)
search_results

['1. 벨파아제 · Good Italian bistro ; 2. 강동반상 · good atmosphere, cost-effective Korean restaurant ; 3. 스키마야 · 라멘, 덮밥 추천 · 덮밥류가 맛있는 집 ; 4. 신불 ...',
 '서울특별시 강동구 상일로5길 8-7 1층. 저장. 전화. 버거비 상일동점. 서울특별시 강동구 천호대로221길 6 1층. 저장. 전화. 오한수우육면가 상일점. 서울 ...',
 [{'position': 1,
   'rating': 4.8,
   'reviews': 103,
   'reviews_original': '(103)',
   'price': '₩20,000~30,000',
   'description': '"평일에도 웨이팅 있는 맛집이니 정말 맛집으로 인정합니다."',
   'lsig': 'AB86z5UFl0mK4kmmAXrfFYIPVOk8',
   'thumbnail': 'https://serpapi.com/searches/68a59fe9fd9b90721358cd3f/images/1d724752a7fe6b9c7d07a9c369de006552804776a7e39b96c6107ea3e995a1313c11c1859202628e.jpeg',
   'place_id': '12730341462943213745',
   'place_id_search': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=kr&google_domain=google.com&hl=ko&ludocid=12730341462943213745&num=3&q=5%ED%98%B8%EC%84%A0+%EC%83%81%EC%9D%BC%EB%8F%99%EC%97%AD+%EC%A3%BC%EB%B3%80%EC%97%90+%EC%9E%88%EB%8A%94+%EB%A7%9B%EC%A7%91',
   'gps_coordinates': {'latitude': 37.557636, 'longitud

In [36]:
# Gemini 2.5 Flash 모델을 사용하여 식당명을 추출합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List
import re
import ast


# 식당 정보를 담는 모델 정의
class Restaurant(BaseModel):
    name: str = Field(description="식당명")


# 여러 식당 정보를 담는 모델 정의
class RestaurantList(BaseModel):
    restaurants: List[Restaurant]


# OutputParser 초기화
parser = PydanticOutputParser(pydantic_object=RestaurantList)

# Gemini 2.5 Flash 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

# 프롬프트 템플릿 생성
prompt_template = """
다음 검색 결과에서 식당명만을 추출해주세요.
각 검색 결과를 분석하여 식당 이름을 정확히 식별하고 JSON 형식으로 출력해주세요.

{format_instructions}

검색 결과:
{search_data}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Gemini 모델을 사용하여 식당명 추출
chain = prompt | model | parser

# search_results를 문자열로 변환하여 모델에 전달
search_data_str = str(search_results)
result_obj = chain.invoke({"search_data": search_data_str})

# 파서로 JSON 출력
import json

json_str = result_obj.model_dump_json(indent=2)
print("=== JSON 출력 ===")
print(json_str)

=== JSON 출력 ===
{
  "restaurants": [
    {
      "name": "벨파아제"
    },
    {
      "name": "강동반상"
    },
    {
      "name": "스키마야"
    },
    {
      "name": "버거비 상일동점"
    },
    {
      "name": "오한수우육면가 상일점"
    },
    {
      "name": "고반식당 상일동역점"
    },
    {
      "name": "송하정스시"
    },
    {
      "name": "쇠나무그릴"
    }
  ]
}


In [37]:
print("\n=== 식당명 목록 ===")
# 보기 좋게 출력
for i, r in enumerate(result_obj.restaurants, 1):
    print(f"{i}. 식당명: {r.name}")
    print("-" * 30)


=== 식당명 목록 ===
1. 식당명: 벨파아제
------------------------------
2. 식당명: 강동반상
------------------------------
3. 식당명: 스키마야
------------------------------
4. 식당명: 버거비 상일동점
------------------------------
5. 식당명: 오한수우육면가 상일점
------------------------------
6. 식당명: 고반식당 상일동역점
------------------------------
7. 식당명: 송하정스시
------------------------------
8. 식당명: 쇠나무그릴
------------------------------


### OutputParser 적용

### gemma3:4b 모델을 사용

- 정상적으로 작동은 하지만, langchain이 아닌 LLMChain을 사용하는 코드로 생성해줌.

In [38]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import json

def analyze_article(article_text):
    """
    기사를 분석하여 주요 내용을 추출하고 JSON 형식으로 반환합니다.

    Args:
        article_text: 기사 텍스트

    Returns:
        JSON 형식의 분석 결과
    """

    # 1. 핵심 정보 추출
    author = "베르나르 베르베르"
    title = "베르나르 베르베르 '작가는 미래 고민해야…고난 헤쳐가는 인류 그렸죠'"
    publication_date = "2025.08.20. 오후 5:22"
    source = "뉴스1"
    novel_title = "<키메라의 땅>"
    main_themes = [
        "미래 고민",
        "고난 헤쳐가는 인류",
        "과학적 상상력",
        "철학적 질문",
        "기후 위기",
        "식량난",
        "전쟁"
    ]
    key_quotes = [
        "작가는 본질적으로 인류를 위해 더 나은 미래가 뭔지 사유하는 사람",
        "극단적인 상황에서도 인류에 대한 해결책을 찾는 유토피아적 작품을 지향한다",
        "인간이 호모사피엔스 단 한 종만 존재한다는 것은 놀라운 일이다.",
        "이 책의 주인공도 우리가 한 종만 있다는 사실이 인간을 각종 외부 환경과 재난으로부터 약하게 만든다고 생각하는 인물",
        "'나도 박쥐처럼 날개가 있다면 어떤 느낌이 들까' 상상하며 즐길 수 있을 것"
    ]

    # 2. 텍스트 요약
    summary = "베르나르 베르베르 작가는 미래에 대한 고민과 인류의 고난 극복에 대한 유토피아적 작품을 지향하며, <키메라의 땅>을 통해 과학적 상상력과 철학적 질문을 결합했습니다. 이 소설은 핵전쟁 이후 황폐해진 지구를 배경으로 인간과 동물의 유전자 조합으로 탄생한 신인류 '키메라'들의 생존 방식을 다루며, 기후 위기, 식량난, 전쟁 등의 현실 문제를 반영합니다. 독자들은 혼종들의 행동을 통해 상상력을 자극하고 미래에 대한 질문을 던지도록 유도합니다."

    # 3. JSON 형식으로 변환
    analysis_result = {
        "author": author,
        "title": title,
        "publication_date": publication_date,
        "source": source,
        "novel_title": novel_title,
        "main_themes": main_themes,
        "key_quotes": key_quotes,
        "summary": summary
    }

    return json.dumps(analysis_result, indent=4, ensure_ascii=False)


# 기사 텍스트 (위 기사 내용)
article_text = """
설지연 기자
베르나르 베르베르 "작가는 미래 고민해야…고난 헤쳐가는 인류 그렸죠"
입력2025.08.20. 오후 5:22 기사원문
... (기사 내용) ...
"""

# 분석 실행
json_output = analyze_article(article_text)
print(json_output)


{
    "author": "베르나르 베르베르",
    "title": "베르나르 베르베르 '작가는 미래 고민해야…고난 헤쳐가는 인류 그렸죠'",
    "publication_date": "2025.08.20. 오후 5:22",
    "source": "뉴스1",
    "novel_title": "<키메라의 땅>",
    "main_themes": [
        "미래 고민",
        "고난 헤쳐가는 인류",
        "과학적 상상력",
        "철학적 질문",
        "기후 위기",
        "식량난",
        "전쟁"
    ],
    "key_quotes": [
        "작가는 본질적으로 인류를 위해 더 나은 미래가 뭔지 사유하는 사람",
        "극단적인 상황에서도 인류에 대한 해결책을 찾는 유토피아적 작품을 지향한다",
        "인간이 호모사피엔스 단 한 종만 존재한다는 것은 놀라운 일이다.",
        "이 책의 주인공도 우리가 한 종만 있다는 사실이 인간을 각종 외부 환경과 재난으로부터 약하게 만든다고 생각하는 인물",
        "'나도 박쥐처럼 날개가 있다면 어떤 느낌이 들까' 상상하며 즐길 수 있을 것"
    ],
    "summary": "베르나르 베르베르 작가는 미래에 대한 고민과 인류의 고난 극복에 대한 유토피아적 작품을 지향하며, <키메라의 땅>을 통해 과학적 상상력과 철학적 질문을 결합했습니다. 이 소설은 핵전쟁 이후 황폐해진 지구를 배경으로 인간과 동물의 유전자 조합으로 탄생한 신인류 '키메라'들의 생존 방식을 다루며, 기후 위기, 식량난, 전쟁 등의 현실 문제를 반영합니다. 독자들은 혼종들의 행동을 통해 상상력을 자극하고 미래에 대한 질문을 던지도록 유도합니다."
}


### gpt-oss:20b 모델을 사용해 다시 제안해 보았다.

In [39]:
# --------------------------------------------------------------
#  1. 필요 라이브러리 설치 (한 번만 실행하면 됨)
# --------------------------------------------------------------
# pip install langchain pydantic openai tiktoken  # openai API 키 필요

# --------------------------------------------------------------
#  2. imports
# --------------------------------------------------------------
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# --------------------------------------------------------------
#  3. 출력 모델 정의 (PydanticOutputParser 사용)
# --------------------------------------------------------------
class SummaryOutput(BaseModel):
    """
    기사 원문에 대한 짧은 한 문장 요약
    """
    summary: str = Field(
        ...,
        description="기사 내용을 한 문장으로 요약한 문장"
    )

# --------------------------------------------------------------
#  4. Prompt 템플릿 정의
# --------------------------------------------------------------
prompt = PromptTemplate(
    template="""
아래는 뉴스 기사 원문입니다.  
아래 기사 내용을 최대한 핵심을 잡아 1~2문장(한 줄)으로 요약해 주세요.  
답변은 JSON 형식으로 반환해야 하며, 반드시 다음 구조를 따라야 합니다:

{schema}

기사 원문:
{article}
""",
    input_variables=["article"],
    partial_variables={"schema": SummaryOutput.schema_json()},
)

# --------------------------------------------------------------
#  5. 모델 + 파서 연결
# --------------------------------------------------------------
# 5-1. OpenAI 모델(예: gpt-4o-mini) 지정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# 5-2. PydanticOutputParser 인스턴스
parser = PydanticOutputParser(pydantic_object=SummaryOutput)

# 5-3. Chain 구성
chain = (
    prompt
    | llm
    | parser
)

# --------------------------------------------------------------
#  6. 예시 기사 입력
# --------------------------------------------------------------
article_text = """
프랑스의 이야기꾼 베르나르 베르베르가 신작 장편소설 <키메라의 땅>을 들고 한국을 찾았다. <개미> <타나토노트> <파피용> 등으로 국내 독자들의 큰 사랑을 받은 그는 이번에도 과학적 상상력과 철학적 질문을 결합한 흡인력 있는 소설을 선보였다.

베르베르 작가는 20일 기자간담회에서 “한국은 제2의 조국과 같은 나라”라며 “한국에선 미래를 향해 나아가는 에너지를 느낀다”고 인사를 건넸다. 그는 “작가는 본질적으로 인류를 위해 더 나은 미래가 뭔지 사유하는 사람”이라며 “극단적인 상황에서도 인류에 대한 해결책을 찾는 유토피아적 작품을 지향한다”고 신작을 소개했다.

<키메라의 땅>은 3차 세계대전 이후 핵전쟁으로 황폐해진 지구를 무대로 한다. 극소수의 인간만 살아남은 곳에서 진화 생물학자인 주인공 알리스 카메러는 인간과 동물의 유전자 조합으로 탄생한 신인류 ‘키메라’를 만들어낸다. 박쥐의 능력을 이어받아 하늘을 나는 에어리얼, 두더지와 결합해 지하에 적응한 디거, 돌고래와 결합해 헤엄치는 능력을 지닌 노틱. 이들은 멸망의 땅에서 옛 인류와 갈등하고 연대하며 새로운 생존 방식을 개척한다. 그의 소설은 단순히 미래적 상상력에 머물지 않는다. 기후 위기, 식량난, 끊임없는 전쟁의 위협 등 우리가 살아가는 현실의 문제를 정면으로 비추며 인류의 미래에 대한 불안과 희망을 동시에 그려냈다.

베르베르 작가는 “인간이 호모사피엔스 단 한 종만 존재한다는 것은 놀라운 일이다. 개미만 해도 1만2000여 종이 존재한다”며 “이 책의 주인공도 우리가 한 종만 있다는 사실이 인간을 각종 외부 환경과 재난으로부터 약하게 만든다고 생각하는 인물”이라고 설명했다. 그는 “실제 과학적 발견을 바탕으로 썼다”며 “우리가 더 멀리 나아갈 수 있다는 걸 공상과학(SF) 장르와 엮어 보여주고 싶었다”고 덧붙였다.

작가는 “독자들이 책에 나오는 혼종들의 행동을 보며 재미를 느끼기를 바란다”며 “‘나도 박쥐처럼 날개가 있다면 어떤 느낌이 들까’ 상상하며 즐길 수 있을 것”이라고 말했다.
"""

# --------------------------------------------------------------
#  7. 실행 & 결과 확인
# --------------------------------------------------------------
if __name__ == "__main__":
    # OPENAI_API_KEY 환경 변수에 키를 저장해 두었거나
    # os.environ["OPENAI_API_KEY"] = "sk-..."
    result = chain.invoke({"article": article_text})
    print(result.summary)


C:\Users\SBA\AppData\Local\Temp\ipykernel_8632\3178174474.py:42: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  partial_variables={"schema": SummaryOutput.schema_json()},


OutputParserException: Failed to parse SummaryOutput from completion {"description": "\uae30\uc0ac \uc6d0\ubb38\uc5d0 \ub300\ud55c \uc694\uc57d", "properties": {"summary": {"description": "\ud504\ub791\uc2a4 \uc791\uac00 \ubca0\ub974\ub098\ub974 \ubca0\ub974\ubca0\ub974\uac00 \uc2e0\uc791 \uc18c\uc124 <\ud0a4\uba54\ub77c\uc758 \ub545>\uc744 \ubc1c\ud45c\ud558\uba70, \uc778\ub958\uc758 \ubbf8\ub798\uc640 \ud604\uc2e4 \ubb38\uc81c\ub97c \ub2e4\ub8ec \uc791\ud488\uc744 \uc18c\uac1c\ud588\ub2e4.", "title": "Summary", "type": "string"}}, "required": ["summary"], "title": "SummaryOutput", "type": "object"}. Got: 1 validation error for SummaryOutput
summary
  Field required [type=missing, input_value={'description': '기사 ...tput', 'type': 'object'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 